## Import libraries

In [1]:
import numpy as np
from keras.models import Model, load_model
from keras.layers import Flatten, Dense, Lambda
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.preprocessing import image

Using TensorFlow backend.


## Set variables and import data

In [2]:
gen = image.ImageDataGenerator()
batch_size = 64

In [3]:
batches = gen.flow_from_directory('task3_data_mega/train',
                                  target_size=(224,224),
                                  class_mode='categorical',
                                  shuffle=True,
                                  batch_size=batch_size)

Found 64000 images belonging to 7 classes.


In [4]:
val_batches = gen.flow_from_directory('task3_data_mega/valid',
                                      target_size=(224,224),
                                      class_mode='categorical',
                                      shuffle=True,
                                      batch_size=batch_size)

Found 16000 images belonging to 7 classes.


## Import, configure, and compile model

In [16]:
resnet = ResNet50(include_top=True, weights='imagenet', input_tensor=None, input_shape=(224,224,3), pooling=None)

x = resnet.layers[-2].output
output_layer = Dense(7, activation='softmax', name='predictions')(x)

resnet2 = Model(inputs=resnet.input, outputs=output_layer)

resnet2.compile(optimizer=Adam(lr=0.00003),
                loss='categorical_crossentropy', metrics=['accuracy'])

## Train and save model

In [17]:
resnet2.fit_generator(batches,
                   steps_per_epoch = batches.samples // batch_size,
                   validation_data = val_batches, 
                   validation_steps = val_batches.samples // batch_size,
                   epochs = 1)

Epoch 1/1
1000/1000 [==============================] - 514s 514ms/step - loss: 0.3958 - acc: 0.8576 - val_loss: 0.4438 - val_acc: 0.8516


In [18]:
resnet2.save('task3_models/resnet.h5')

## Load model, generate predictions, and output results

In [3]:
resnet2 = load_model('task3_models/resnet.h5')

In [8]:
test = gen.flow_from_directory('task3_data_mega/test_final',
                                target_size=(224,224),
                                class_mode='categorical',
                                shuffle=False,
                                batch_size=batch_size)

Found 1512 images belonging to 1 classes.


In [9]:
f_names = np.array(test.filenames)
predictions = np.array(resnet2.predict_generator(test))
resnet_predictions = np.column_stack((f_names, predictions))
np.savetxt("resnet_predictions_final_raw.csv", resnet_predictions, delimiter=",", fmt="%s")